In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib.parse as urlparse
import re
import random
import webbrowser
import requests
from re import sub
from decimal import Decimal
import numpy as np
import pandas as pd

### Extract name list from kaggle data

In [2]:
df_kaggle = pd.read_csv("./kaggle_data/train.csv",sep=",")

name_of_movie = df_kaggle.original_title

lan_list =  df_kaggle.original_language
en_list = [x for x in lan_list if x!="en"]
en_indx = df_kaggle[df_kaggle.original_language=="en"]["id"]

### Search the corresponding first-week box office

In [3]:
def find_box_office(movie_id=None,box=None):
    if movie_id == None:
        return None
    else:
        base_url_movie = "https://www.boxofficemojo.com/movies/"
        param_movie = {"page":"weekend","id":movie_id}
        r_ = requests.get(base_url_movie, params=param_movie)
        url_ = r_.url
        html_ = urlopen(url_).read()
        soup_ = BeautifulSoup(html_,'lxml')
        sub_ = soup_.find_all('font',{'size':re.compile('2'), 'face':None})
        # title_ = soup_.find_all("font",{"face":re.compile("Verdana"),"size":re.compile("[4-6]")})
        # title_ = title_[0].get_text()
        for k in range(len(sub_)):
            if ('$' in sub_[k].get_text()):
                box = sub_[k].get_text()
                return Decimal(sub(r'[^\d.]', '', box))
        return box

In [4]:
def find_webID(movie_name,url_list,webID=None):
    for i in range(len(url_list)):
        if(movie_name in url_list[i].get_text() or url_list[i].get_text() in movie_name):
            webID = urlparse.parse_qs(urlparse.urlparse(url_list[i]["href"]).query)
            return webID["id"]
    return None

In [5]:
url_search = "https://www.boxofficemojo.com/search/"
# name_of_movie = ["Insight","Control Room","A Mighty Wind"]

counter = 0 # comment the counter out when establishing the data set

for i in en_indx:
    param = {'q': name_of_movie[i-1]}
    r_search = requests.get(url_search,params=param)
    url_search = r_search.url
    html_search = urlopen(url_search).read()
    soup_search = BeautifulSoup(html_search,"lxml")

    sub_url_search = soup_search.find_all("a",{"href":re.compile("/movies/\?id")})
    
    webID = find_webID(name_of_movie[i-1],sub_url_search)
    
    # par_search_id = urlparse.parse_qs(urlparse.urlparse(sub_url_search[1]["href"]).query)
    
    box_office = find_box_office(webID)
    
    print(i," <",name_of_movie[i-1],">"," the first week box office is: ",box_office)
    
    ####### comment this out when establishing the data set #######
    counter+=1
    if counter==100:
        break
    ###############################################################

1  < Hot Tub Time Machine 2 >  the first week box office is:  5963324
2  < The Princess Diaries 2: Royal Engagement >  the first week box office is:  22956453
3  < Whiplash >  the first week box office is:  135388
6  < Pinocchio and the Emperor of the Night >  the first week box office is:  None
7  < The Possession >  the first week box office is:  None
8  < Control Room >  the first week box office is:  27125
9  < Muppet Treasure Island >  the first week box office is:  7906689
10  < A Mighty Wind >  the first week box office is:  2112140
11  < Rocky >  the first week box office is:  None
12  < Revenge of the Nerds II: Nerds in Paradise >  the first week box office is:  7900879
13  < American Beauty >  the first week box office is:  861531
14  < Be Cool >  the first week box office is:  23450212
15  < Minority Report >  the first week box office is:  35677125
17  < Red Eye >  the first week box office is:  16167662
18  < The Invisible Woman >  the first week box office is:  31948
19  